In [1]:
#%reload_ext Cython

In [2]:
#import typing
#import copy
#import pdb
#from IPython.display import clear_output
#import numpy as np
#import time
#from colorama import Fore
#import sys
#import cProfile
#from multiprocessing import Pool
#import chess 
#import chess.uci
#import tensorflow as tf
#import random
#import boardclass
import test
import boardclass

In [3]:
path = "../extracted/good_games.txt"
#boardclass.file_to_move_lists(path,move_size=5)
test.compare_boards_with_file(path)

At 2, 5, expected -6, got 0. Previous moves: ['e4e5', 'f6d5', 'f3d5', 'b7d5', 'g3e4', 'd5e4', 'd4e4', 'h6h5', 'c2c3', 'f7f5'], reached 70 out of 78. Move is e5f6
